In [ ]:
import pandas as pd
from datetime import datetime, timedelta
version_in = 'V0_1'
version_out = 'V1_1314'
path_gen = "/home/pol/Escritorio/INVERBIS/2024/CancerColon/"
path_1 = path_gen + "Data/processed_data/CancerColon_semi"+str(version_out)+".csv"
colon = pd.read_csv(path_1, sep= ';')

In [ ]:
print(colon.shape)
colon.columns

In [ ]:
colon.groupby('Tipo vía rápida')['NASI seudonimizado'].nunique()

In [ ]:
colon['Actividad'].unique().tolist()

In [ ]:
import numpy as np
# Función para convertir a float si es necesario
def convertir_a_float(valor):
    if isinstance(valor, np.int64):
        return int(valor)
    else:
        return valor  # Si ya es float, lo dejamos como está
    

def conditions_surgery_quimio(cirugia_index, quimioterapia_index, trace):
    # Comprobar las condiciones posibles
    if np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('El primer valor es nulo y el segundo no')
        indice_cirugia_quimioterapia = quimioterapia_index
    elif not np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('El primer valor no es nulo y el segundo sí')
        indice_cirugia_quimioterapia = cirugia_index
    elif not np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('Ambos valores no son nulos')
        # Mantener la actividad que ocurra primero
        indice_cirugia_quimioterapia = min(cirugia_index, quimioterapia_index)
    elif np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('Ambos valores son nulos')
        indice_cirugia_quimioterapia = len(trace)
    
    return indice_cirugia_quimioterapia

def tc_abdominal(df):
    return df[df['Actividade CAPNOR'].str.contains('TC') & df['Actividade CAPNOR'].str.contains('ABDOM')]['Actividade CAPNOR'].unique().tolist()


lista_TCAbdominal = tc_abdominal(colon)

In [ ]:
def appointment_post_biopsia(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')
    df["num_citas_med_interna (Colono-Cirugia/quimio)"] = 0
    df["num_citas_oncologia (Colono-Cirugia/quimio)"] = 0
    df["num_citas_med_cir_general (Colono-Cirugia/quimio)"] = 0
    df["num_citas_digestivo (Colono-Cirugia/quimio)"] = 0
    df["num_tac_abdominal (Colono-Cirugia/quimio)"] = 0
    df["num_colonoscopia (Colono-Cirugia/quimio)"] = 0
    df["num_colonoscopia_VDR (Colono-Cirugia/quimio)"] = 0

    df["num_citas_med_interna (Cirugia/quimio-final)"] = 0
    df["num_citas_oncologia (Cirugia/quimio-final)"] = 0
    df["num_citas_med_cir_general (Cirugia/quimio-final)"] = 0
    df["num_citas_digestivo (Cirugia/quimio-final)"] = 0
    df["num_tac_abdominal (Cirugia/quimio-final)"] = 0
    df["num_colonoscopia (Cirugia/quimio-final)"] = 0
    df["num_colonoscopia_VDR (Cirugia/quimio-final)"] = 0
    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
        trace = trace.reset_index(drop=True)
        # Obtener el índice de la biopsia diagnóstica
        biopsia_indice = trace[trace['Actividad'] == 'Biopsia diagnóstica'].index.min()

        # Obtener el índice de la primera ocurrencia de 'Cirugía' y 'Quimioterapia' después de la biopsia
        cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
        quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
        cirugia_index = convertir_a_float(cirugia_index)
        quimioterapia_index = convertir_a_float(quimioterapia_index)
        print(cirugia_index)
        print(quimioterapia_index)
        #logica entre cirugia y quimio
        indice_cirugia_quimioterapia = conditions_surgery_quimio(cirugia_index, quimioterapia_index)
        print(biopsia_indice,indice_cirugia_quimioterapia)

        #Contadores entre biosia y cirugia/quimio
        rango_indices = trace.index[biopsia_indice:indice_cirugia_quimioterapia]  # Puedes ajustar el rango según tu necesidad

        trace["num_citas_med_interna (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Medicina interna').sum()
        trace["num_citas_oncologia (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Oncología').sum()
        trace["num_citas_med_cir_general (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Cirugía General').sum()
        trace["num_citas_digestivo (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Digestivo').sum()
        trace["num_tac_abdominal (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita TAC Abdominal').sum()
        trace["num_colonoscopia (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('COLONOSCOPIA').sum()
        trace["num_colonoscopia_VDR (Colono-Cirugia/quimio)"] = trace.loc[rango_indices, 'Actividad'].eq('COLONOSCOPIA VIA RAPIDA').sum()

        #Contadores entre cirugia/quimio y el final
        rango_indices = trace.index[indice_cirugia_quimioterapia:]  # Puedes ajustar el rango según tu necesidad

        trace["num_citas_med_interna (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Medicina interna').sum()
        trace["num_citas_oncologia (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Oncología').sum()
        trace["num_citas_med_cir_general (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Cirugía General').sum()
        trace["num_citas_digestivo (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita en Digestivo').sum()
        trace["num_tac_abdominal (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('Cita TAC Abdominal').sum()
        trace["num_colonoscopia (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('COLONOSCOPIA').sum()
        trace["num_colonoscopia_VDR (Cirugia/quimio-final)"] = trace.loc[rango_indices, 'Actividad'].eq('COLONOSCOPIA VIA RAPIDA').sum()


        traces.append(trace)

    return pd.concat(traces)


resultados = appointment_post_biopsia(colon) 

In [ ]:
def counters_pre_biopsia(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')
    df["NUM_LAXANTES"] = 0

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
    
        # Convertir la columna FECHA a tipo datetime
        trace['FECHA'] = pd.to_datetime(trace['FECHA'], format= '%d-%m-%Y %H:%M:%S')
        # Encontrar la fecha de la actividad 'Biopsia'
        fecha_biopsia = trace.loc[trace['Actividad'] == 'Biopsia diagnóstica', 'FECHA'].iloc[0]
        # Restar 6 meses a la fecha de la biopsia
        fecha_limite = fecha_biopsia - pd.DateOffset(months=6)
        # Filtrar los registros dentro del rango de tiempo
        registros_filtrados = trace[(trace['FECHA'] >= fecha_limite) & (trace['FECHA'] <= fecha_biopsia)]
        # Contar el número de veces que aparece 'laxante' en el campo 'medicamentos'
        num_laxante_formador = registros_filtrados['Subg. químico terapéutico ATC disp'].value_counts().get('LAXANTES FORMADORES DE VOLUMEN', 0)
        num_laxante_contacto = registros_filtrados['Subg. químico terapéutico ATC disp'].value_counts().get('LAXANTES DE CONTACTO', 0)
        num_laxante_osmotico = registros_filtrados['Subg. químico terapéutico ATC disp'].value_counts().get('LAXANTES OSMÓTICOS', 0)
        print(f"""Número de veces que aparece 'laxante' desde al menos 6 meses hacia atrás desde la actividad 'Biopsia':\n
              {num_laxante_formador}
              {num_laxante_contacto}
              {num_laxante_osmotico}""")
        print(f"Numero de laxantes totales {num_laxante_formador + num_laxante_contacto + num_laxante_osmotico}")
        trace["NUM_LAXANTES"] = num_laxante_formador + num_laxante_contacto + num_laxante_osmotico
        traces.append(trace)

    return pd.concat(traces)


df_res = counters_pre_biopsia(colon) 

In [ ]:
def counters_quimio(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')
    df["NUM_QUIMIOS"] = 0

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
    
        # Convertir la columna FECHA a tipo datetime
        trace['FECHA'] = pd.to_datetime(trace['FECHA'], format= '%d-%m-%Y %H:%M:%S')
        # Encontrar la fecha de la actividad 'Biopsia'
        fecha_biopsia = trace.loc[trace['Actividad'] == 'Biopsia diagnóstica', 'FECHA'].iloc[0]
        # Filtrar los registros dentro del rango de tiempo
        registros_filtrados = trace[trace['FECHA'] >= fecha_biopsia]
        # Contar el número de veces que aparece 'laxante' en el campo 'medicamentos'
        num_quimios = registros_filtrados['Actividad'].value_counts().get('Quimioterapia', 0)
        # num_laxante_contacto = registros_filtrados['Subg. químico terapéutico ATC disp'].value_counts().get('LAXANTES DE CONTACTO', 0)
        # num_laxante_osmotico = registros_filtrados['Subg. químico terapéutico ATC disp'].value_counts().get('LAXANTES OSMÓTICOS', 0)
        print(f"Número de veces que aparece quimioterapia desde la actividad 'Biopsia':{num_quimios}")
        trace["NUM_QUIMIOS"] = num_quimios
        traces.append(trace)

    return pd.concat(traces)


df_quimios = counters_quimio(colon) 

In [ ]:
df_quimios_filter = df_quimios.query("NUM_QUIMIOS >= 59")
df_quimios_filter['NASI seudonimizado'].unique()


In [ ]:
colon_tolon = colon[colon['Actividad'] == 'Cirugia']
colon_tolon['GNA'].unique().tolist()

In [ ]:
colon['GNA'].unique().tolist()

In [ ]:
def surgery_activities(df, attrib_name):
    traces = []
    grouper_3 = df.groupby('NASI seudonimizado')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == attrib_name:
                if trace['Tipo intervención'].values[index] == 'PROGRAMADA': 
                    trace['Actividad'].values[index] = "Cirugia Programada"
                elif trace['Tipo intervención'].values[index] == 'URXENTE': 
                    trace['Actividad'].values[index] = "Cirugia Urgente"
                                
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #renombre cirugia URXENTE
    traces_fix = traces_fix.rename(columns={"Cirugia PROGRAMADA": "Cirugia Programada", "Cirugia URXENTE": "Cirugia Urgente"})
    #eliminar actividad cita normal
    return traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == attrib_name].index)

surgery_colon = surgery_activities(colon, 'Cirugia')

In [ ]:
surgery_colon['Actividad'].unique().tolist()

In [ ]:
groupon = colon.groupby('NASI seudonimizado')

for name, trace in groupon:
    print(name)
    print(trace['Actividad'].values.tolist())
    print('\n')

In [ ]:

def create_initial_activity(df):
    lista = ['Cita en Medicina interna', 'Cita en Oncología', 'Cita en Cirugía General', 'Cita en Digestivo']
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:

        # Crear una máscara booleana para identificar las filas donde 'Actividad' es 'biopsia'
        mask_biopsia = trace['Actividad'] == 'Biopsia diagnóstica'
        # Crear una máscara booleana para identificar las filas donde 'Actividad' está en la lista
        mask_lista = trace['Actividad'].isin(lista)
        # Crear una máscara booleana que marque las filas que deben eliminarse
        mask_eliminar = mask_biopsia.cumsum() == 0
        # Aplicar la máscara para eliminar las filas anteriores a 'biopsia' excepto las que están en la lista
        trace = trace[~(mask_eliminar & ~mask_lista)]
        trace = trace.reset_index(drop=True)

        primer_indice = trace.index.to_list()[0]
        print(primer_indice)
        biopsia_indice = trace[trace['Actividad'] == 'Biopsia diagnóstica'].index.min()
        print(biopsia_indice)

        if (biopsia_indice - primer_indice) > 1:
            print('resta', biopsia_indice)
            trace = trace.iloc[biopsia_indice-1:]
        traces.append(trace)

    return pd.concat(traces)           

initial_fix = create_initial_activity(colon)  

In [ ]:
import numpy as np
# Función para convertir a float si es necesario
def convertir_a_float(valor):
    if isinstance(valor, np.int64):
        return int(valor)
    else:
        return valor  # Si ya es float, lo dejamos como está
    

def conditions_surgery_quimio(cirugia_index, quimioterapia_index):
    # Comprobar las condiciones posibles
    if np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('El primer valor es nulo y el segundo no')
        indice_cirugia_quimioterapia = quimioterapia_index
    elif not np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('El primer valor no es nulo y el segundo sí')
        indice_cirugia_quimioterapia = cirugia_index
    elif not np.isnan(cirugia_index) and not np.isnan(quimioterapia_index):
        print('Ambos valores no son nulos')
        # Mantener la actividad que ocurra primero
        indice_cirugia_quimioterapia = min(cirugia_index, quimioterapia_index)
    elif np.isnan(cirugia_index) and np.isnan(quimioterapia_index):
        print('Ambos valores son nulos')
        indice_cirugia_quimioterapia = len(trace)
    
    return indice_cirugia_quimioterapia
    


In [ ]:
def appointment_post_biopsia(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
        trace = trace.reset_index(drop=True)
        # Obtener el índice de la biopsia diagnóstica
        biopsia_indice = trace[trace['Actividad'] == 'Biopsia diagnóstica'].index.min()

        # Obtener el índice de la primera ocurrencia de 'Cirugía' y 'Quimioterapia' después de la biopsia
        cirugia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Cirugia'].index.min()
        quimioterapia_index = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Quimioterapia'].index.min()
        cirugia_index = convertir_a_float(cirugia_index)
        quimioterapia_index = convertir_a_float(quimioterapia_index)
        print(cirugia_index)
        print(quimioterapia_index)
        #logica entre cirugia y quimio
        indice_cirugia_quimioterapia = conditions_surgery_quimio(cirugia_index, quimioterapia_index)
        print(biopsia_indice,indice_cirugia_quimioterapia)

        interna = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Medicina interna'].index
        Oncologia = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Oncología'].index
        general = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Cirugía General'].index
        digestivo = trace[trace.index > indice_cirugia_quimioterapia][trace['Actividad'] == 'Cita en Digestivo'].index
        medico = trace[trace.index > biopsia_indice][trace['Actividad'] == 'Visita médico familia'].index

        # cirugia_index = convertir_a_float(cirugia_index)
        # quimioterapia_index = convertir_a_float(quimioterapia_index)
        list_to_lists = []
        list_to_lists.extend(interna)
        list_to_lists.extend(Oncologia)
        list_to_lists.extend(general)
        list_to_lists.extend(digestivo)
        list_to_lists.extend(medico)
        print(list_to_lists)
    
        trace = trace.drop(list_to_lists)
        traces.append(trace)

    return pd.concat(traces)


resultados = appointment_post_biopsia(colon) 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def counters_pre_biopsia(df):
    # Agrupar por el identificador de traza
    groupon = df.groupby('NASI seudonimizado')

    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        print(name)
    
        # Convertir la columna FECHA a tipo datetime
        df['FECHA'] = pd.to_datetime(df['FECHA'])
        # Encontrar la fecha de la actividad 'Biopsia'
        fecha_biopsia = df.loc[df['actividad'] == 'Biopsia', 'FECHA'].iloc[0]
        # Restar 6 meses a la fecha de la biopsia
        fecha_limite = fecha_biopsia - pd.DateOffset(months=6)
        # Filtrar los registros dentro del rango de tiempo
        registros_filtrados = df[(df['FECHA'] >= fecha_limite) & (df['FECHA'] <= fecha_biopsia)]
        # Contar el número de veces que aparece 'laxante' en el campo 'medicamentos'
        num_laxante = registros_filtrados['medicamentos'].value_counts().get('laxante', 0)
        print("Número de veces que aparece 'laxante' desde al menos 6 meses hacia atrás desde la actividad 'Biopsia':", num_laxante)
        traces.append(trace)

    return pd.concat(traces)


df_res = counters_pre_biopsia(df) 

In [ ]:
resultados_cirugia = resultados[resultados['Actividad'] == 'Cirugia']
grouponi = resultados_2.groupby('NASI seudonimizado')

for name, trace in grouponi:
    print(name)
    print(trace['Actividad'].values.tolist())
    print('\n')

In [ ]:
def only_first_quimio(df):
    groupon = df.groupby('NASI seudonimizado')
    traces = []
    # Iterar sobre cada grupo
    for name, trace in groupon:
        
        trace = trace.reset_index(drop=True)
        indices_quimioterapia = trace.loc[trace['Actividad'] == 'Quimioterapia'].index
        if not indices_quimioterapia.empty:
            print(name)
            delete_index = indices_quimioterapia[1:]
            print(delete_index)
            trace = trace.drop(delete_index)
        traces.append(trace)
    return pd.concat(traces)

resultados_2 = only_first_quimio(resultados)


In [ ]:
def cita_extracion(df):
    # Filtrar el DataFrame para obtener solo las filas donde la actividad es 'Cita' o 'extraccion'
    df_filtered = df[df['Actividad'].isin(['Cita', 'extraccion'])]

    # Contar cuántas veces aparece cada valor de fecha asociada para cada ID
    counts = df_filtered.groupby(['NASI seudonimizado', 'Actividad', 'FECHA']).size().reset_index(name='counts')

    # Filtrar los resultados para encontrar los ID donde la fecha asociada a 'Cita' es igual a la fecha asociada a 'extraccion'
    matching_ids = counts[counts['Actividad'] == 'Data cita'].merge(counts[counts['Actividad'] == 'DATA EXTRACCIÓN'], on='NASI seudonimizado', suffixes=('_Cita', '_extraccion'))
    matching_ids = matching_ids[matching_ids['FECHA_Cita'] == matching_ids['FECHA_extraccion']]

    # Mostrar los ID resultantes
    print(matching_ids['NASI seudonimizado'].unique())


cita_extracion(colon)


In [ ]:
def tc_abdominal(df):
    return df[df['Actividade CAPNOR'].str.contains('TC') & df['Actividade CAPNOR'].str.contains('ABDOM')]['Actividade CAPNOR'].unique().tolist()


lista_TCAbdominal = tc_abdominal(colon)
lista_TCAbdominal

In [ ]:
colon['Actividad'].unique().tolist()

In [ ]:
def waiting_list_duration(df, enter_list, exit_list):
    traces = []
    df['Duracion Lista Espera'] = 0
    grouper_3 = df.groupby('NASI seudonimizado')
    #pasar ambas columnas de fecha a datatime
    df[enter_list] = pd.to_datetime(df[enter_list], format= '%d-%m-%Y %H:%M:%S')
    df[exit_list] = pd.to_datetime(df[exit_list], format= '%d-%m-%Y %H:%M:%S')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == enter_list: #! trazas con lista de espera de entrada
                trace['Duracion Lista Espera'] = (trace[exit_list] - trace[enter_list]).dt.days
                trace['Duracion Lista Espera'] = trace['Duracion Lista Espera'].fillna(0)
                trace['Duracion Lista Espera'] = trace['Duracion Lista Espera'].astype(int) 
        
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #eliminar actividades lista espera
    traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == enter_list].index)
    traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == exit_list].index)
    return traces_fix




colon_hardcodeado_2 = waiting_list_duration(colon, 'Data inclusión', 'Data remate')

In [ ]:
colon_hardcodeado_2[['NASI seudonimizado', 'FECHA', 'FECHA_FIN','Actividad', 'Duracion Lista Espera']].head(10)
colon_hardcodeado_2['Duracion Lista Espera'].unique()

In [ ]:
#! Añadir actividad cita con apellido del tipo de Cita y dejar unicamente los relacionados con COLONOSCOPIA u otros importantes
#'GNA','Actividade CAPNOR', 'Código Prestación', 'Prestación','Tipo vía rápida',
colon_citas = colon.query("Actividad == 'Data prescrición'")
capnor = colon_citas.groupby('Actividade CAPNOR')['NASI seudonimizado'].nunique().to_dict()
gna = colon_citas.groupby('GNA')['NASI seudonimizado'].nunique().to_dict()
# Ordenar el diccionario de mayor a menor por sus valores
capnor_ordenado = dict(sorted(capnor.items(), key=lambda item: item[1], reverse=True))
gna_ordenado = dict(sorted(gna.items(), key=lambda item: item[1], reverse=True))

for i, j in zip(capnor_ordenado.keys(),capnor_ordenado.values()):
    print(f'{i} -> {j}')

In [ ]:
#Quedarme con las actividades de tipo cita si es de tipo colonoscopia - Separar entre colonoscopia normal y VDR
#PASOS
#   1. Colocar a actividad Cita el apellido de su campo CAPNOR
#   2. Solo mantener aquellas que el apellido sea Colonoscopia o VDR u otras importantes
#   3. Con los datos en bruto, sacar ids traza que tienen colonoscopia y si son normal o VDR y colocar posteriormente como atributos

def surname_Appointments(df, attrib_name):
    traces = []
    ids_colonoscopy = []
    df['VDR'] = False
    df['Colonoscopia'] = False
    lista = ['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA']
    grouper_3 = df.groupby('NASI seudonimizado')

    for name, trace in grouper_3:
        for index in range(len(trace['Actividad'])):

            if trace['Actividad'].values[index] == attrib_name:
                if trace['Actividade CAPNOR'].values[index] in lista: 
                    trace['Actividad'].values[index] = 'Cita ' + trace['Actividade CAPNOR'].values[index]
                    trace['Colonoscopia'] = True
                    if trace['Tipo vía rápida'].values[index] == "VIA RAPIDA COLON              ":
                        trace['VDR'] = True

                    
        traces.append(trace)
    traces_fix = pd.concat(traces)
    #eliminar actividad cita normal
    return traces_fix.drop(traces_fix.loc[traces_fix['Actividad'] == attrib_name].index)


colon_hardcodeado = surname_Appointments(colon, 'Data prescrición')

In [ ]:
colon_hardcodeado['Actividad'].unique()
colon_hardcodeado['Colonoscopia'].unique()

In [ ]:
colon_hardcodeado['NASI seudonimizado'].nunique()

In [ ]:
colon_hardcodeado['VDR'].unique()

In [ ]:
colon.groupby('Tipo vía rápida')['NASI seudonimizado'].nunique()

In [ ]:
colon_hardcodeado.groupby('VDR')['NASI seudonimizado'].nunique()

In [ ]:
def no_colonoscopy(df):
    filtro_sin = ~df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    filtro_con = df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    # print(df.shape)
    # print(df['NASI seudonimizado'].nunique())
    colon_colonoscopia = df[filtro_con]
    # print(colon_colonoscopia.shape)
    # print(colon_colonoscopia['NASI seudonimizado'].nunique())
    lista_validos = colon_colonoscopia['NASI seudonimizado'].unique().tolist()

    colon_sin_colonoscopia = df[filtro_sin]
    # print(colon_sin_colonoscopia.shape)
    # print(colon_sin_colonoscopia['NASI seudonimizado'].nunique())
    return lista_validos

In [ ]:
lista  = no_colonoscopy(colon)
print(lista)
coloner = colon[colon['NASI seudonimizado'].isin(lista)]

In [ ]:
coloner['NASI seudonimizado'].nunique()

In [ ]:
colon['Actividad'].unique()

In [ ]:
def filter_dataApa(trace, value):
    trace['FECHA'] = pd.to_datetime(trace['FECHA'], format = '%d-%m-%Y %H:%M:%S')
    # Fecha límite para el filtrado, también como datetime
    limit_date = pd.to_datetime(value, format= '%d-%m-%Y %H:%M:%S')

    limit_date_minus_10s = limit_date - pd.Timedelta(seconds=10)
    trace_filtered = trace[trace['FECHA'] >= limit_date_minus_10s] 
    print(value)
    print(trace_filtered['Actividad'].unique())
    print('\n')
    return trace_filtered


def crear_dead_bolean(trace):
    trace['Defuncion'] = True
    return trace


def modify_dead_date(trace):
    max_date = trace['FECHA'].max()
    new_date = max_date + timedelta(days=30)
    new_date = pd.to_datetime(new_date, format= '%d-%m-%Y %H:%M:%S')
    # Cambiar las fechas asociadas a 'Defunción'
    trace.loc[trace['Actividad'] == 'Defunción', ['FECHA', 'FECHA_FIN']] = new_date
    return trace

In [ ]:
#recorrer traza a traza
# y para cada traza: mirar la fecha de extracción de ese paciente
#                  -> filtrar filas de ese trace superiores a esa fecha (DATA_EXTRACT)

def filter_extraction(df):

    df['FECHA'] = pd.to_datetime(df['FECHA'], format= '%d-%m-%Y %H:%M:%S')
    df['FECHA_FIN'] = pd.to_datetime(df['FECHA_FIN'], format= '%d-%m-%Y %H:%M:%S')
    grouper = df.groupby('NASI seudonimizado')

    dates, traces = [], []
    dates_dict = {}
    for name, trace in grouper:
        trace['Defuncion'] = False
        if 'Prueba laboratorio (APA)' in trace['Actividad'].unique().tolist():
            print(name)
            df = trace.query(" Actividad == 'Prueba laboratorio (APA)'")
            value = df['FECHA'].values[0]
            dates.append(value)
            dates_dict[name] = value
            
            #!boleano defunciones
            if 'Defunción' in trace['Actividad'].unique().tolist():
                trace = crear_dead_bolean(trace)
                trace = modify_dead_date(trace)
                
            print('FILTRADO!!!\n')

            traces.append(filter_dataApa(trace, value))

    return pd.concat(traces)

colon_filtered = filter_extraction(colon)

In [ ]:
# def boolAttrib_to_EmptyAttrib(df, atrrib):
#     #recorro cada traza del df
#     grouper = df.groupby('NASI seudonimizado')
#     traces = []
#     for name, trace in grouper:
#         values = trace[attrib].dropna().astype(str).unique().tolist()
#         trace[attrib]  = ', '.join(values)
#         traces.append(trace)
        

#     return pd.concat(traces)
    

# attribs = ['Prioridade', 'GNA', 'Actividade CAPNOR', 'Prestación', 'Tipo vía rápida',	'Detalle tipo actividade','Tipo programación']
# for attrib in attribs:
#     colon_filtered = boolAttrib_to_EmptyAttrib(colon_filtered, attrib)


In [ ]:
colon_filtered = colon_filtered[colon_filtered['Actividad'] != 'Preescripcion consultas y pruebas']

In [ ]:
colon_filtered['Actividade CAPNOR'].unique()

In [ ]:
def filter_important_drugs(df):
    important_drugs = ['LAXANTES OSMÓTICOS','LAXANTES FORMADORES DE VOLUMEN', 'ENEMAS']
    return df[df['Subg. químico terapéutico ATC disp'].isin(important_drugs)]

In [ ]:
def drugs_info(df):
    # #! Función que conteara el numero de veces que va a buscar medicamentos, y cuanto le cuesta en total estos medicamentos
    grouper_2 = colon_filtered.groupby('NASI seudonimizado')

    colon_filtered['Nº de compras farmaceuticas'] = 0
    colon_filtered['Coste total medicamentos'] = 0
    traces = []
    for name, trace in grouper_2: # Prezo PVP nomen disp
        cont = 0 
        try:
            cont = trace['Actividad'].value_counts()['Dispensación Farmacia Oncologica']
            print(cont)
            trace['Nº de compras farmaceuticas'] = cont
            trace['Coste total medicamentos'] = trace['Prezo PVP nomen disp'].sum()
        except KeyError:
            trace['Nº de compras farmaceuticas'] = 0
            trace['Coste total medicamentos'] = 0
        traces.append(trace)
    
    return pd.concat(traces)

colon_filtered  = drugs_info(colon_filtered)

In [ ]:
id = 'edf8341be887aa3da8126840589d7cb9'
colon_filtered_prueba = colon_filtered[colon_filtered['NASI seudonimizado'] == id]
colon_filtered_prueba['Actividad'].unique()

In [ ]:
def filter_important_drugs(df):
    for group_id, group_data in df.groupby('ID'):
        if not group_data['Algun_medicamento_importante'].any():
            # Eliminar las filas donde la actividad sea 'Dispensación medicamento ATC'
            df = df.drop(group_data[group_data['Actividad'] == 'Dispensación medicamento ATC'].index)
    return df


dfss = filter_important_drugs(colon_filtered)


In [ ]:
colon_filtered[['NASI seudonimizado', 'FECHA', 'FECHA_FIN','Actividad', 'Actividade CAPNOR','Subg. químico terapéutico ATC disp', 'Nº de compras farmaceuticas', 'Coste total medicamentos']]

In [ ]:
iterator = colon_filtered.groupby('NASI seudonimizado')

only_traces = []
for name, trace in iterator:
    activ = trace['Actividad'].unique().tolist()
    if len(activ) == 1:
        only_traces.append(name)

print(only_traces)

In [ ]:
colon_filtered['Defuncion'].value_counts()

In [ ]:
colon_filtered['FECHA'].max()

In [ ]:
colon_filtered

In [ ]:
# Eliminar '\n' y '\r' en valores y cabeceras
#colon_filtered = colon_filtered.replace({r'\r': '', r'\n': ''}, regex=True)

In [ ]:
colon_filtered.to_csv(path_gen+"Data/processed_data/CancerColon_basico_filtered"+str(version_out)+".csv", date_format="%d-%m-%Y %H:%M:%S", index=False,  encoding="UTF-8", sep = ';')

In [ ]:
no_colonoscopy(colon_filtered)